In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc1


In [14]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cpu"

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = Sampler(res, train_index, test_index, null_mask)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:1.227653 auc:0.5015
epoch:  20 loss:0.293133 auc:0.9424
epoch:  40 loss:0.217822 auc:0.9478
epoch:  60 loss:0.202691 auc:0.9505
epoch:  80 loss:0.196917 auc:0.9526
epoch: 100 loss:0.192962 auc:0.9542
epoch: 120 loss:0.188073 auc:0.9564
epoch: 140 loss:0.183178 auc:0.9584
epoch: 160 loss:0.182064 auc:0.9590
epoch: 180 loss:0.180232 auc:0.9594
epoch: 200 loss:0.179262 auc:0.9597
epoch: 220 loss:0.178444 auc:0.9599
epoch: 240 loss:0.181122 auc:0.9601
epoch: 260 loss:0.177049 auc:0.9604
epoch: 280 loss:0.176350 auc:0.9605
epoch: 300 loss:0.176886 auc:0.9607
epoch: 320 loss:0.174559 auc:0.9608
epoch: 340 loss:0.173073 auc:0.9611
epoch: 360 loss:0.173068 auc:0.9616
epoch: 380 loss:0.171339 auc:0.9620
epoch: 400 loss:0.169155 auc:0.9624
epoch: 420 loss:0.170897 auc:0.9625
epoch: 440 loss:0.167152 auc:0.9630
epoch: 460 loss:0.168450 auc:0.9640
epoch: 480 loss:0.164823 auc:0.9641
epoch: 500 loss:0.163575 auc:0.9642
epoch: 520 loss:0.163092 auc:0.9647
epoch: 540 loss:0.160362 auc

In [20]:
true_datas.reset_index(drop=True).to_csv("true_gdsc1.csv")
predict_datas.reset_index(drop=True).to_csv("pred_gdsc1.csv")